In [1]:
#pip3 install google.colab
#from google.colab import drive
#drive.mount('/content/drive', force_remount=True)

###  Libraries

In [2]:
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.colors as mcolors

import torch

import os
import sys

from Autoencoder import Autoencoder, CD_Autoencoder
from Datasets.Datasets_Functions import *
from Visualization import Visualization
from Evaluations.Evaluation import Evaluator
from General_Functions import General_Functions

### Global Variables

In [3]:
path_to_module = './' # 'C:\\Users\\PAPASOFT INC\\Desktop\\SOFT_SIL'
sys.path.append(path_to_module)
os.environ['OMP_NUM_THREADS'] = '6'

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)

### Visualization

In [4]:
visualization = Visualization()

### ***LOAD DATASET***

In [5]:
hashmap_path = path_to_module + 'Datasets/'
hashmap = get_hashmap(hashmap_path)
print(hashmap.keys())
dataset_name = "r100"
#update_outer_hashmap('emnist_balanced_digits',{'batch_size':256, 'n_clusters':10, 'module_name': 'Datasets.Datasets'}, hashmap_path)
dataset_properties = hashmap[dataset_name]
print(dataset_properties)
batch_size = dataset_properties['batch_size'] = 256
dataloader, input_dim, data_np, labels = function_get_dataset(dataset_name, dataset_properties)
print('Data_Shape is:', input_dim)
print('Batches Number is:', len(dataloader))

dict_keys(['tcga', 'emnist_balanced_letters', 'emnist_mnist', 'emnist_balanced_digits', 'dermatology', 'ecoil', 'iris', 'rings', 'wine', 'australian', 'moons', 'squeezed_gauss', 'gauss_densities', 'pendigits', 'fashionmnist', '3dspheres', '20_newsgroups', 'coil20', 'cifar10', 'stl10', 'r15', 'r3', 'olivetti_faces', 'r100', '10x73k', 'usps', 'kmnist', 'reuters_4', 'hars'])
{'batch_size': 1024, 'n_clusters': 4, 'module_name': 'Datasets.Datasets'}


[nltk_data] Downloading package punkt to C:\Users\PAPASOFT
[nltk_data]     INC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\PAPASOFT
[nltk_data]     INC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Data_Shape is: 100
Batches Number is: 40


In [6]:
if(False):
    print(data_np.shape)
    #data_np = np.reshape(data_np, (len(data_np), 28 * 28))
    kmeans = KMeans(n_clusters=dataset_properties['n_clusters'], n_init=10).fit(data_np)
    evaluator = Evaluator()
    evaluator.evaluate_model(data_np, labels, kmeans.labels_)
    

### Parameters

In [7]:
# Latent Dimension, Number of Channels and Negative Slope
latent_dim = 10
n_clusters = dataset_properties['n_clusters'] # = 6
# Batch Size and Number of Clusters
batch_size = dataset_properties['batch_size'] #= 1024
negative_slope = 0

# Pre-Training Epochs and Learning Rate
n_pret_epochs = 100
pret_lr = 1e-3

# Lamdas, Training Epochs and Learning Rate
n_epochs = 100
lr = 5e-4
sil_lambda = 0.01
entr_lambda = 0.01

kmeans_initialization = True
pretrain = True
is_MLP_AE = True

### Create Autoencoder Model

In [8]:
if is_MLP_AE:
    autoencoder = Autoencoder(device=device, n_clusters=n_clusters, input_dim=input_dim, latent_dim=latent_dim, negative_slope=negative_slope)
else:
    autoencoder = CD_Autoencoder(device=device, n_clusters=n_clusters, input_dim=input_dim, latent_dim=latent_dim, negative_slope=negative_slope)
autoencoder.set_general_training_variables(dataloader=dataloader, batch_size=batch_size)
autoencoder.set_pretraining_variables(n_pret_epochs=n_pret_epochs, pret_lr=pret_lr)
autoencoder.set_training_variables(n_epochs=n_epochs, lr=lr, sil_lambda=sil_lambda, entr_lambda=entr_lambda)
autoencoder.set_path_variables(path_to_module=path_to_module, dataset_name=dataset_name)
autoencoder.set_path()
autoencoder = autoencoder.to(device)

In [9]:
autoencoder

Autoencoder(
  (encoder_model): Sequential(
    (0): Linear(in_features=100, out_features=500, bias=True)
    (1): LeakyReLU(negative_slope=0, inplace=True)
    (2): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Linear(in_features=500, out_features=500, bias=True)
    (4): LeakyReLU(negative_slope=0, inplace=True)
    (5): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Linear(in_features=500, out_features=2000, bias=True)
    (7): LeakyReLU(negative_slope=0, inplace=True)
    (8): BatchNorm1d(2000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Linear(in_features=2000, out_features=10, bias=True)
    (10): Tanh()
    (11): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (cluster_model): Sequential(
    (0): RBF()
  )
  (decoder_model): Sequential(
    (0): Linear(in_features=10, out_features=2000, bias=True)
    (1): LeakyReLU(negative_s

### Pretrain Autoencoder

In [10]:
if(pretrain):
    autoencoder.pretrain_autoencoder()
    autoencoder.save_pretrained_weights()
else:
    model_save_path = autoencoder.data_dir_path + '/Weigths/autoencoder_weights.pth'
    autoencoder.load_state_dict(torch.load(model_save_path))

Epoch: 0, Loss: 3.401958
Epoch: 1, Loss: 1.667900
Epoch: 2, Loss: 0.994101
Epoch: 3, Loss: 0.780274
Epoch: 4, Loss: 0.552981
Epoch: 5, Loss: 0.566718
Epoch: 6, Loss: 0.610756
Epoch: 7, Loss: 0.894002
Epoch: 8, Loss: 0.571277
Epoch: 9, Loss: 0.565692
Epoch: 10, Loss: 0.567350
Epoch: 11, Loss: 0.364141
Epoch: 12, Loss: 0.380103
Epoch: 13, Loss: 0.210606
Epoch: 14, Loss: 0.167939
Epoch: 15, Loss: 0.206399
Epoch: 16, Loss: 0.166258
Epoch: 17, Loss: 0.116307
Epoch: 18, Loss: 0.123428
Epoch: 19, Loss: 0.107431
Epoch: 20, Loss: 0.105181
Epoch: 21, Loss: 0.095329
Epoch: 22, Loss: 0.099944
Epoch: 23, Loss: 0.097724
Epoch: 24, Loss: 0.100457
Epoch: 25, Loss: 0.092633
Epoch: 26, Loss: 0.096737
Epoch: 27, Loss: 0.077420
Epoch: 28, Loss: 0.122596
Epoch: 29, Loss: 0.145651
Epoch: 30, Loss: 0.119124
Epoch: 31, Loss: 0.082732
Epoch: 32, Loss: 0.076208
Epoch: 33, Loss: 0.077081
Epoch: 34, Loss: 0.070682
Epoch: 35, Loss: 0.076956
Epoch: 36, Loss: 0.070644
Epoch: 37, Loss: 0.090425
Epoch: 38, Loss: 0.072

In [11]:
#if(False):
data, latent_data, labels = autoencoder.get_latent_data()
kmeans = KMeans(n_clusters=n_clusters, n_init=100).fit(latent_data)
evaluator = Evaluator()
evaluator.evaluate_model(data, labels, kmeans.labels_)

(0.9381, 0.9381, 0.8380540513327964, 0.8482333723561085, 0)

In [12]:
evaluator.EVA()

AttributeError: 'Evaluator' object has no attribute 'EVA'

### Initialize the clustering layer using k-means

In [ ]:
n_inits = 100
if kmeans_initialization: autoencoder.kmeans_initialization(n_inits)

### Train Autoencoder

In [ ]:
#autoencoder.set_training_variables(n_epochs=100, lr=5e-4, sil_lambda=0.01, entr_lambda=0.01)
latent_data, labels, clustering = autoencoder.train_autoencoder()

In [ ]:
cluster_centers = autoencoder.get_cluster_centers().cpu().detach().numpy()
data, latent_data, labels = autoencoder.get_latent_data()
General_Functions().save_excel(autoencoder.data_dir_path, autoencoder.df_eval)

In [ ]:
visualization.plot(latent_data, labels, clustering, cluster_centers, autoencoder.data_dir_path)

In [ ]:
#visualization.plot_tsne(latent_data, labels, clustering, cluster_centers, autoencoder.data_dir_path)

In [ ]:
data, similarity, labels = autoencoder.get_similarity_data()
print(labels)
images = General_Functions().find_images_to_plot(30, 36, data, similarity, labels)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.ticker import FixedLocator
import matplotlib.colors as mcolors

from sklearn.manifold import TSNE

from openpyxl import load_workbook,Workbook
import os.path

from General_Functions import General_Functions

def plot_collage(images, num_x_images, num_y_images, image_size, data_dir_path):
    
        # Create a blank canvas for the collage
        collage_size = (image_size[0] * 10, image_size[1] * 10)

        fig, axes = plt.subplots(num_x_images, num_y_images, figsize=(image_size[0], image_size[1]), facecolor = 'black')

        for i in range(num_x_images):
            for j in range(num_y_images):
                index = i * num_y_images + j
                image = images[index].reshape(image_size[0], image_size[1])
                
                axes[i, j].imshow(image, cmap='gray')
                axes[i, j].axis('off')

        plt.subplots_adjust(wspace=0, hspace=0)
        plt.savefig(data_dir_path + "/Experiments/Collage.png", facecolor = 'white')
        plt.show()
plot_collage(images, 6, 10, (28, 28), autoencoder.data_dir_path)
from PIL import Image

# Open an image
image_path = "path/to/your/image.jpg"
original_image = Image.open(image_path)

# Set the desired size in pixels
new_size = (width_in_pixels, height_in_pixels)

# Resize the image
resized_image = original_image.resize(new_size)

# Save or display the resized image
resized_image.save("path/to/your/resized/image.jpg")
resized_image.show()